In [1]:
import requests, re
from bs4 import BeautifulSoup
from IPython.display import HTML, display

import pandas as pd

# https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss
# goes to page 278

# first page of thread:
# store data in lists --> pandas dataframe

# THREAD NAME
# <h3 class="post-title"><a href="/about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same#post-1305941">I can&#039;t live without my dad - anyone feel the same?</a></h3>

# THREAD ADDRESS
# page_link

# USERNAME
# <div class="field field-name-ds-user-picture field-type-ds field-label-hidden">
# <div class="field-items"><div class="field-item even">
# <a href="/about-cancer/cancer-chat/users/paulus">

# POST DATE
# <span class="post-created hidden-xs">22 Jun 2020 16:35</span>

# POST TEXT
# <div class="field field-name-field-harmony-text field-type-text-long field-label-hidden">

# POST IS A REPLY TO POST_ID
# <span class="post-is-reply-to">22 Jun 2020 21:04 in response to <a href="/about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same#post-1305941" class="active">bras1548</a></span>

# POST ID
# <a name="post-1305941"></a>


#### FUNCTION: SCRAPE A THREAD PAGE

In [106]:
# FUNCTION: SCRAPE A THREAD PAGE
def page_scraper_cancer_research(page_link):

    req = requests.get(page_link)
    soup = BeautifulSoup(req.text, 'lxml')
    
    # THREAD ADDRESS 
    thread_address_list = []
    thread_address = soup.find("link",{"rel":"canonical"})['href']
    thread_address = page_link
    #print(thread_address)

    # THREAD NAME
    thread_name_list = []
    thread_group = soup.find("h3", {"class": "post-title"})
    thread_name = thread_group.find("a").text
    #print(thread_name)

    # POST ID
    post_id_list = []
    post_id_group = soup.findAll("a", {"name": re.compile(r".*")})

    for post_id_g in post_id_group:
        if "post-" in post_id_g["name"]:
            post_id_list.append(post_id_g["name"])
            #print(post_id_g["name"])

    # POST USERNAME
    username_list = []
    user_group = soup.findAll("div", {"class": "field field-name-ds-user-picture field-type-ds field-label-hidden"})
    
    for user_g in user_group:    
        username = user_g.find("a")["href"]
        username = username.replace("/about-cancer/cancer-chat/users/", "")
        username_list.append(username)

        # populate THREAD ADDRESS, THREAD NAME lists
        thread_address_list.append(thread_address)
        thread_name_list.append(thread_name)
        #print(username)

    # POST DATE
    date_list = []
    date_group = soup.findAll("span", {"class": "post-created hidden-xs"})

    for date_g in date_group:
        date = date_g.text
        date_list.append(date)
        #print(date)

    # POST TEXT
    text_list = []
    text_group = soup.findAll("div", {"class": "field field-name-field-harmony-text field-type-text-long field-label-hidden"})

    for text_g in text_group:
        text_tag = text_g.find("div").find("div")
        text = text_tag.text
        text_list.append(text)
        #print(text_tag.text, "\n")

    # POST IS A REPLY TO
    reply_to_list = []
    reply_group = soup.findAll("span", {"class": "post-is-reply-to"})

    # add "N/A only to first posts in thread
    
    reply_group_len = len(reply_group)
    text_group_len = len(text_group)
    
    while reply_group_len < text_group_len:  # if CHANGED TO while
        reply_to_list.append("N/A")
        reply_group_len += 1
        
    for reply_g in reply_group:
        reply_to_id = reply_g.find("a")['href']
        reply_to_id = "https://www.cancerresearchuk.org" + reply_to_id
        reply_to_id = reply_to_id.replace(thread_address + "#", "")
        reply_to_list.append(reply_to_id)
        #print(reply_to_id)

    # COLUMN LISTS INTO A LIST OF ROWS
    new_rows = []

    for j in range(len(thread_name_list)):
        row_list = [thread_name_list[j], thread_address_list[j], username_list[j],
                    post_id_list[j], date_list[j], text_list[j], reply_to_list[j]]

        new_rows.append(row_list)
    
    return new_rows



#### FUNCTION: CHECK FOR "NEXT" PAGE IN THREAD

In [41]:
# FUNCTION: CHECK FOR "NEXT" PAGE IN THREAD

# <li class="cr-pagination__next">
# <a href="/about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same?page=1">Next</a></li>

def next_page_cancer_research(page_link):

    req = requests.get(page_link)
    soup = BeautifulSoup(req.text, 'lxml')

    next_link = soup.find("li", {"class":"cr-pagination__next"})

    if next_link:
        next_link = next_link.find("a")['href']
        next_link = "https://www.cancerresearchuk.org/" + next_link
        #print(next_link)
        return next_link

    else:
        #print("last page")
        return None

print(next_page_cancer_research("https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same?page=4#post-1515206"))
print(next_page_cancer_research("https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same"))



None
https://www.cancerresearchuk.org//about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same?page=1


In [83]:
# TEST PAGE SCRAPER FUNCTION

page_link = "https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same?page=4"
new_rows = page_scraper_cancer_research(page_link)


In [84]:
for row in new_rows:
    print(row[0])
    print(row[1])
    print(row[2])
    print(row[3])
    print(row[4])
    print(row[5])
    print(row[6], "\n")

I can't live without my dad - anyone feel the same?
https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same?page=4
bras1548
post-1514356
7 Feb 2021 13:08
Hi SamC93,Thanks for your thoughts. Like you, I find it extremely unhelpful when people say those things. Here are the things that I find completely useless:- "What would your father want for you?"- "He'll always be with you"- "You have to live your life now"- "It will get better with time"I don't WANT to live without my dad. He WAS my life. Or at least the key part of my life. He was the sunshine in my world. The idea of letting him go and living my own life (without him) is appalling to me. I'm glad you're finding strength somewhere 
https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same#post-1513781 

I can't live without my dad - anyone feel the same?
https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/i-

#### FUNCTION: SCRAPE ONE THREAD

In [107]:
# FUNCTION: SCRAPE ONE THREAD

row_list = []
page_link = "https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same"

def thread_scraper_cancer_research(page_link):

    new_rows = page_scraper_cancer_research(page_link)
    next_link = next_page_cancer_research(page_link)
    
    #for row in new_rows:  # REMOVE THIS LATER
    #    print(row[0])
    #    print(row[1])
    #    print(row[2])
    #    print(row[3])
    #    print(row[4])
    #    print(row[5])
    #    print(row[6], "\n")
        
    
    if next_link is not None:
        new_rows.extend(thread_scraper_cancer_research(next_link))

    return new_rows

In [86]:
# TEST: SCRAPE ONE THREAD

page_link = "https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same"

new_rows = thread_scraper_cancer_research(page_link)
print(len(new_rows))

64


#### FUNCTION: GET THREAD LINKS FROM PAGE

In [46]:
# FUNCTION: GET THREAD LINKS FROM PAGE

forum_link = "https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss"

def thread_link_scraper_cancer_research(forum_link):

    thread_links = []
    req = requests.get(forum_link)
    soup = BeautifulSoup(req.text, 'lxml')

    # <div class="tl-cell views-field views-field-nothing" >
    # <a href="/about-cancer/cancer-chat/thread/has-anyone-tried-bereavement-counselling">

    thread_link_group = soup.find_all("div", {"class":"tl-cell views-field views-field-nothing"})
    
    for thread_link_gr in thread_link_group:    
        new_thread_link = thread_link_gr.find('a')['href']
        thread_links.append("https://www.cancerresearchuk.org" + new_thread_link)
        #print("https://www.cancerresearchuk.org" + new_thread_link, "\n")
        
    return thread_links
    


#### FUNCTION: CHECK FOR "NEXT" PAGE IN FORUM

In [47]:
# FUNCTION: CHECK FOR "NEXT" PAGE IN FORUM

def next_forum_page_cancer_research(forum_link):
    
    req = requests.get(forum_link)
    soup = BeautifulSoup(req.text, 'lxml')

# <li class="next last"><a title="Go to next page" href="
    next_link = soup.find("li", {"class":"next last"})

    if next_link:
        next_link = next_link.find("a")['href']
        next_link = "https://www.cancerresearchuk.org" + next_link
        #print(next_link)
        return next_link

    else:
        #print("last page")
        return None

print(next_forum_page_cancer_research("https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss"))
print(next_forum_page_cancer_research("https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss?page=278"))



https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss?page=1
None


#### FUNCTION: SCRAPE ALL THREAD LINKS ON FORUM

In [87]:
# FUNCTION: SCRAPE ALL THREAD LINKS ON FORUM

def thread_links_cancer_research(forum_link):
    
    thread_links = []
    current_page = forum_link
    new_links = thread_link_scraper_cancer_research(current_page)
    thread_links.extend(new_links)
    
    next_link = next_forum_page_cancer_research(current_page)
    #print(next_link)
    
    if next_link is not None:
        thread_links.extend(thread_links_cancer_research(next_link))
        
    return thread_links




In [13]:
# TEST FUNCTION: SCRAPE ALL THREAD LINKS ON FORUM

#forum_link = "https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss"
#thread_links = thread_links_cancer_research(forum_link)


In [127]:
# OPEN FILE WITH SAVED 'COPING WITH LOSS' LINKS

thread_links = pd.read_csv(r'/Users/Sofia/Desktop/cancer_research_coping_loss_links.csv', delimiter="\t")
thread_links = thread_links["0"].values.tolist()
print(len(thread_links))


#test_list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

#for i in test_list[0:3]:
#    print(i)
    
#print("\n")

#for i in test_list[3:6]:
#    print(i)
    
#print("\n")

#for i in test_list[6:]:
#    print(i)

2786


In [108]:
# DEBUGGING PROBLEM PAGE

bug_link = "https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/confused-30"
#bug_link = "https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/seeing-signs-from-deceased-loved-ones?page=7"

bug_rows = thread_scraper_cancer_research(bug_link)
#bug_rows = page_scraper_cancer_research(bug_link)

In [110]:
#for line in bug_rows:
#    print(line[0])
#    print(line[1])
#    print(line[2])
#    print(line[3])
#    print(line[4])
#    print(line[5])
#    print(line[6], "\n")

In [124]:
# RUN 'SCRAPE THREAD' ON THREAD_LINKS

#coping_with_loss_data = [] # DO NOT RESET !!

count = 2001

for thread in thread_links[2000:]:
    
    print(count, thread)

    new_rows = thread_scraper_cancer_research(thread)    
    coping_with_loss_data.extend(new_rows)
    
    count += 1


2001 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/would-anyone-be-willing-to-share-the-story-of-their-loss
2002 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/devastated-and-so-angry
2003 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/whats-the-point
2004 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/am-i-wrong-0
2005 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/lost-my-dad-april-2017
2006 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/losing-a-forum-buddy-meerkat65-jo
2007 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/struggling-with-the-loss-of-my-mum
2008 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/nsclc-coping-with-loss-no-reply
2009 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/really-struggling-0
2010 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/will-this-guilt-go-away
2011 https://www.cance

2089 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/where-do-our-loved-ones-go
2090 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/missing-my-dad-like-crazy
2091 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/coping-with-mums-death
2092 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/saying-goodbye
2093 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/new-22
2094 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/mums-gone-flat-is-cleared-feeling-lost
2095 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/im-struggling-without-my-husband
2096 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/feeling-alone-5
2097 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-mum-passed-away-0
2098 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/oesophageal-cancer-14
2099 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/d

2174 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/practical-advice-please-form-filling-following-bereavement
2175 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/3-weeks-on
2176 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/miss-my-mum-so-much
2177 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/no-real-emotion-after-my-fathers-death
2178 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/christmas-0
2179 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/lost-my-mum-a-week-ago-and-i-am-not-feeling-my-emotions
2180 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/secondary-breast-cancer-in-the-liver
2181 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-husband-has-gone
2182 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/after-those-who-disapear-what-about-xmas-invites
2183 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thre

2259 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/will-i-ever-be-able-to-say-goodbye-mum
2260 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/i-have-lost-my-young-husband
2261 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-lovely-mommy-passed-away-2-months-ago
2262 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/i-feel-lost
2263 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-dad-just-died-a-month-ago
2264 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/funeral-0
2265 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/no-words
2266 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/dealing-with-grief-0
2267 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/so-sudden
2268 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-dads-cancer-fight
2269 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-dad-is

2349 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/losing-my-dad-6
2350 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/missing-mum-3
2351 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/mum-has-gone
2352 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/happy-birthday
2353 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/im-now-a-widow
2354 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/a-month-on-and-i-still-feel-numb
2355 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-dad-had-lung-cancer
2356 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/lost-my-16-year-old-brother
2357 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/loss-of-my-sister
2358 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/lost-my-lovely-mum
2359 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/losing-my-dad-5
2360 https://www.can

2437 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/advice-17
2438 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/at-last-some-good-news
2439 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/ive-lost-my-dad
2440 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/rip-david-bowie
2441 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/hope-everyone-had-a-great-new-year
2442 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/this-is-soooo-hard
2443 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/after-the-funeral
2444 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/how-to-continue-without-him
2445 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/bereavement-counselling
2446 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/lost-my-grandad-2-months-a-go-god-i-miss-him
2447 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thr

2526 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/gone-but-not-forgotton
2527 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/depressed
2528 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/i-feel-worse-at-the-weekend
2529 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-mum-died-about-4-months-ago-im-15
2530 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-mam-has-weeks-if-not-days
2531 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-family-deaths
2532 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/good-bye-dad-never-forgotten-finding-it-hard-to-live-out-y
2533 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-lovely-dad-patchie
2534 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/loss-of-my-grandma
2535 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/loss-of-my-dad-0
2536 https://www.cancerresearchuk.org/a

2614 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/helping-someone-cope
2615 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/lost-4
2616 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/feeling-lost-2
2617 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/would-you-like-to-set-up-a-tribute-fund
2618 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/struggling-to-cope-with-loss
2619 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/mum-passed-this-morning
2620 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/cant-cope-2
2621 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/mum-has-terminal-cancer
2622 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-dad-24
2623 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/lost-my-dad-4-days-ago
2624 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/a
2625 https://www.cancer

2702 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/how-to-celebrate-her-birthday-if-shes-not-here
2703 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/25th-november-2013-day-2
2704 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/mum-reunited-with-dad-now-5-months-later
2705 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/just-lost-my-mum
2706 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/i-just-want-my-mam
2707 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/moving-on-what-do-i-do-now
2708 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/when-will-it-be-easier
2709 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/having-a-bad-week
2710 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/is-someone-else-in-the-same-boat-as-me-or-am-i-alone
2711 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/good-days-and-bad-days
2712 ht

2784 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/bereavement-0
2785 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/this-is-life-but-not-as-i-know-it
2786 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/its-so-hard-to-carry-on


In [126]:
# SAVE PRELIMINARY DATA

column_names = ['Thread_name', 'Thread_address', 'Username', "Post_id",
                                  "Date", "Post_text", "Reply_to_id"]

cancer_research_coping_w_loss_prelim = pd.DataFrame(coping_with_loss_data, columns = column_names)  

cancer_research_coping_w_loss_prelim.to_csv(r'/Users/Sofia/Desktop/cancer_research_coping_loss.csv', sep ='\t', index = False, header=True)




In [125]:
print(len(coping_with_loss_data))

# print(len(thread_links))
    
#for line in coping_with_loss_data:
#    print(line[0])
#    print(line[1])
#    print(line[2])
#    print(line[3])
#    print(line[4])
#    print(line[5])
#    print(line[6], "\n")
    

19180


In [181]:
# SAVE 'COPING WITH LOSS' FORUM LINKS TO FILE

#print(len(thread_links))

#for thread in thread_links:
#    print(thread)

cancer_research_coping_w_loss_links = pd.DataFrame(thread_links)
cancer_research_coping_w_loss_links.to_csv(r'/Users/Sofia/Desktop/cancer_research_coping_loss_links.csv', sep ='\t', index = False, header=True)


In [137]:
# GLOBAL database for 
# https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss

column_names = ['Thread_name', 'Thread_address', 'Username', "Post_id",
                                  "Date", "Post_text", "Reply_to_id"]

cancer_research_coping_w_loss = pd.DataFrame(new_rows, columns = column_names)  
print (cancer_research_coping_w_loss)

cancer_research_coping_w_loss.to_csv(r'/Users/Sofia/Desktop/cancer_research_coping_loss_test.csv', sep ='\t', index = False, header=True)


                                          Thread_name  \
0   I can't live without my dad - anyone feel the ...   
1   I can't live without my dad - anyone feel the ...   
2   I can't live without my dad - anyone feel the ...   
3   I can't live without my dad - anyone feel the ...   
4   I can't live without my dad - anyone feel the ...   
5   I can't live without my dad - anyone feel the ...   
6   I can't live without my dad - anyone feel the ...   
7   I can't live without my dad - anyone feel the ...   
8   I can't live without my dad - anyone feel the ...   
9   I can't live without my dad - anyone feel the ...   
10  I can't live without my dad - anyone feel the ...   
11  I can't live without my dad - anyone feel the ...   
12  I can't live without my dad - anyone feel the ...   
13  I can't live without my dad - anyone feel the ...   
14  I can't live without my dad - anyone feel the ...   
15  I can't live without my dad - anyone feel the ...   
16  I can't live without my dad